The following imported .py file contains all of the needed packages and functions for this .ipynb file. The explantion of the functions are also shown in the .py file.

In [3]:
from MINOR_Stage_Functions import *

The following paths are set to the labels and images of the train en validation files, provided to us. This needs to be changed according to the users correct location.

In [4]:
path_train_labels = r"C:\Users\tsboe\OneDrive\Python - programming\PowerMeter2.v1i.yolov8 1\train\labels"
path_train_images = r"C:\Users\tsboe\OneDrive\Python - programming\PowerMeter2.v1i.yolov8 1\train\images"
path_test_labels = r"C:\Users\tsboe\OneDrive\Python - programming\PowerMeter2.v1i.yolov8 1\valid\labels"
path_test_images = r"C:\Users\tsboe\OneDrive\Python - programming\PowerMeter2.v1i.yolov8 1\valid\images"

This code loads and processes label data from the .txt files in the path_train_labels, extracting the first columns of each file, and converting it to integers and storing it in a list.

In [5]:
os.chdir(path_train_labels)

train_y_single = [np.loadtxt(text)[:,0].astype(int)
    for text in sorted_directory_listing(directory=path_train_labels)
    if text.endswith('.txt')]

Where as this code loads and processes image data from the .png files in the path_train_images, extracting the rows with pixel values and ensuring that the images are ready to be used in the model. All of the steps that are being taken on the background can be seen in the .py file.

In [6]:
os.chdir(path_train_images)

train_images_single = []

for img in sorted_directory_listing(path_train_images):
    if img.endswith('.jpg') and img != 'Identiefier':
        train_images_single.append(slice_rotate_image(image_reading(img), height=1.5, length=7, delta_height=0.25, delta_length=0.23, digit_width=0.625))

Here the code does the following steps:
1) Iterates through all images in train_images_single and their corresponding labels in train_y_single.
2) Normalizes image pixel values to a range of 0 to 1, this makes it easier for the model to handle. Also reduces the needed computional power needed to run/train the model.
3) Stores the processed images and their corresponding labels in separate lists.
4) Converts lists into NumPy arrays for further use in training the model.

In [7]:
digit_img_train_divider = []
digit_number_train_divider = []

for i in range(len(train_images_single)):
    for index,image in enumerate(train_images_single[i]):
      image = image/255.0
      digit_img_train_divider.append(image.tolist())
      digit_number_train_divider.append(train_y_single[i][index])
      
digit_img_train_divider = np.array(digit_img_train_divider)
digit_number_train_divider = np.array(digit_number_train_divider)

The above proces is repeated with the test images and labels.

In [8]:
os.chdir(path_test_labels)

test_y_single = [np.loadtxt(text)[:,0].astype(int)
    for text in sorted_directory_listing(directory=path_test_labels)
    if text.endswith('.txt')]

In [9]:
os.chdir(path_test_images)

test_images_single = []
for img in sorted_directory_listing(path_test_images):
    if img.endswith('.jpg') and img != 'Identiefier':
        test_images_single.append(slice_rotate_image(image_reading(img), height=1.5, length=7, delta_height=0.2, delta_length=0.23, digit_width=0.625))

In [10]:
digit_img_test_divider = []
digit_number_test_divider = []

for i in range(len(test_images_single)):
    for index,image in enumerate(test_images_single[i]):
      image = image/255.0
      digit_img_test_divider.append(image.tolist())
      digit_number_test_divider.append(test_y_single[i][index])
      
digit_img_test_divider = np.array(digit_img_test_divider)
digit_number_test_divider = np.array(digit_number_test_divider)

The checkpoint variable will be used in the callback function while fitting the model. In each epoch this check point wil safe the model with the highest validation accuracy as smart_meter_model.keras. Change the file path so the model gets safed in the right directory.

In [11]:
checkpoint = ModelCheckpoint(filepath=r'C:\programeren\jupiter_python\minor\case\smart_meter_model.keras', 
                             monitor='val_accuracy', 
                             mode='max', 
                             save_best_only=True)

This code defines a Keras Sequential model designed for image classification. The model processes grayscale images with dimensions of 50x30 pixels. It employs a convolutional neural network (CNN) architecture with measures to reduce overfitting, such as dropout layers and kernel regularization. The primary purpose of the model is to classify the input images into one of 10 classes, corresponding to the digits 0 through 9.

The model performs three convolutional operations to extract features from the image:
Conv2D layers apply filters of increasing size (32, 64, 128) with a kernel size of 2x2 and ReLU activation.
After each convolution, MaxPooling2D layers with a 2x2 pool size reduce the spatial dimensions, retaining the most important features.
BatchNormalization layers normalize the output of each convolution, improving training stability and speed.
Flattening:

After the convolutional layers, the 2D feature maps are flattened into a 1D vector, preparing the data for the dense layers.

The flattened vector is passed through four fully connected (dense) layers, each with:
128 neurons and ReLU activation for non-linear transformations.
Kernel regularization (L2) to penalize large weights and prevent overfitting.
Dropout layers with a rate of 0.05 to randomly deactivate some neurons during training, further reducing overfitting.
Output Layer:

The final layer has 10 neurons, each corresponding to a digit class, with a softmax activation function to output probabilities for each class.

The model is compiled with:
* Adam optimizer: An adaptive learning rate optimization algorithm.
* SparseCategoricalCrossentropy loss: Used for multi-class classification with integer labels.
* Accuracy metric: To evaluate the model's performance during training and validation.

The model is trained using the fit method:
* Training data: digit_img_train_divider (images) and digit_number_train_divider (labels).
* Validation data: digit_img_test_divider and digit_number_test_divider for assessing performance on unseen data.
* Callbacks: The checkpoint callback (assumed to be defined elsewhere) saves the model at specified intervals during training.
* Epochs: The model is trained for 300 epochs, allowing it to converge.

In [12]:
model = tf.keras.models.Sequential([    
    Conv2D(32, (2, 2), activation='relu', input_shape=(50, 30, 1)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Flatten(),
    Dense(128,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    Dropout(0.05),
    Dense(128,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    Dropout(0.05),
    Dense(128,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    Dropout(0.05),
    Dense(128,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    Dropout(0.05),
    Dense(10,activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.summary()

# from tensorflow.keras.callbacks import ProgbarLogger

model.fit(digit_img_train_divider, 
          digit_number_train_divider,
          validation_data=(digit_img_test_divider,  digit_number_test_divider),
          callbacks = checkpoint,
          epochs=300,
          )

c:\Users\tsboe\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 49, 29, 32)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 24, 14, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 23, 13, 64)     │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 11, 6, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 11, 6, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 5, 128)     │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 5, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5, 2, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257,002 (1003.91 KB)

 Trainable params: 256,554 (1002.16 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/300
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.7180 - loss: 1.4814 - val_accuracy: 0.2196 - val_loss: 2.7999
Epoch 2/300
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9241 - loss: 0.7551 - val_accuracy: 0.0641 - val_loss: 3.1641
Epoch 3/300
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9531 - loss: 0.6278 - val_accuracy: 0.2163 - val_loss: 2.6782
Epoch 4/300
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9641 - loss: 0.5439 - val_accuracy: 0.1010 - val_loss: 2.8315
Epoch 5/300
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9689 - loss: 0.4944 - val_accuracy: 0.2276 - val_loss: 2.6379
Epoch 6/300
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9780 - loss: 0.4381 - val_accuracy: 0.4631 - val_loss: 1.5393
Epoch 7/300
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9797 - loss: 0.3978 - val_accuracy: 0.8958 - val_loss: 0.6160
Epoch 8/300
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9698 - loss: 0.3997 - val_accuracy: 0.

In [177]:
model =load_model(r'C:\programeren\jupiter_python\minor\case\smart_meter_model.keras')

In [178]:
model.evaluate(digit_img_test_divider,  digit_number_test_divider, verbose=2)

20/20 - 1s - 60ms/step - accuracy: 0.9952 - loss: 0.0303


[0.030275698751211166, 0.995192289352417]